In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!pip install evaluate

In [3]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset
from evaluate import load  # ✅ الحل الجديد
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
!pip install sentencepiece

In [5]:
import sentencepiece  # تأكد أن التحميل تم

model_ckpt = "csebuetnlp/mT5_multilingual_XLSum"

# حمل التوكنـايزر
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_ckpt)

# حمل النموذج الفعلي
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_mT5_multilingual_XLSum = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

model.safetensors:  34%|###4      | 1.21G/3.54G [00:00<?, ?B/s]

In [7]:
import pandas as pd
dataset = pd.read_excel('Text_summarization_dataset.xlsx')
dataset.head()

,Column1,Column2
0,summary,text
1,تناول الفاكهة والخضراوات في موسمها. تعرف على أ...,يكون سعر الفاكهة والخضراوات في موسم إنباتها أق...
2,فضل خيارات الأطعمة الأرخص ثمنا. تباطأ في استهل...,الأطعمة الصحية ليست باهظة الثمن بالضرورة، بل ف...
3,ازرع كل ما يمكنك من خضراوات وفاكهة. اطه بنفسك ...,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...
4,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع. التزم ب...,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...


In [8]:
from sklearn.model_selection import train_test_split


train_data, temp_data = train_test_split(dataset, test_size=0.3, random_state=42)

val_data, test_data = train_test_split(temp_data, test_size=2/3, random_state=42)


print(f"Train: {len(train_data)}\nValidation: {len(val_data)}\nTest: {len(test_data)}")

Train: 20535
Validation: 2933
Test: 5868


In [9]:
pipe = pipeline("summarization", model=model_ckpt)


C:\Users\Admin\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


In [10]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    # تأكد أن كل عنصر نصي
    inputs = [str(text) for text in examples["text"]]
    targets = [str(summary) for summary in examples["summary"]]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=64,
            padding="max_length",
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [11]:
train_data = train_data.rename(columns={"Column1": "summary", "Column2": "text"})
val_data = val_data.rename(columns={"Column1": "summary", "Column2": "text"})
test_data = test_data.rename(columns={"Column1": "summary", "Column2": "text"})

train_data = train_data[["summary", "text"]]
val_data = val_data[["summary", "text"]]
test_data = test_data[["summary", "text"]]  # هذا أصلاً تمام

print(train_data.columns)
print(val_data.columns)
print(test_data.columns)


Index(['summary', 'text'], dtype='object')
Index(['summary', 'text'], dtype='object')
Index(['summary', 'text'], dtype='object')


In [12]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

In [13]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20535 [00:00<?, ? examples/s]

C:\Users\Admin\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2933 [00:00<?, ? examples/s]

In [14]:
import transformers
print(transformers.__version__)  # نتأكد أنه 4.30 أو أعلى

4.52.4


In [16]:
import accelerate
print(accelerate.__version__)


1.7.0


In [17]:
from transformers import Seq2SeqTrainingArguments

model.gradient_checkpointing_enable()

# عدّل الـ training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="no",  # نوقف التقييم أثناء التدريب لتوفير الذاكرة
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,  # كبداية
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    fp16=True  # استخدام النصف دقة لتوفير الذاكرة
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

In [19]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
500,2.538700
1000,2.337700
1500,2.240800
2000,2.168900
2500,2.198800
3000,2.141400
3500,2.102300
4000,2.045800
4500,2.082900
5000,2.009100


C:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--csebuetnlp--mT5_multilingual_XLSum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\Lib\site-packages\transformers\modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 84, 'num_beams': 4, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=10268, training_loss=2.1025842087808453, metrics={'train_runtime': 35560.7289, 'train_samples_per_second': 0.577, 'train_steps_per_second': 0.289, 'total_flos': 2.462241231470592e+16, 'train_loss': 2.1025842087808453, 'epoch': 1.0})

In [108]:
# تثبيت Gradio أول مرة
!pip install gradio -q

# استيراد المكتبات
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# تحميل النموذج المدرب
model_path = "C:/Users/Admin/Downloads/results/checkpoint-10268"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# دالة التلخيص
def summarize_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True).to(model.device)
    summary_ids = model.generate(
      **inputs,
      max_length=256,
      min_length=60,
      num_beams=6,
      top_k=50,
      top_p=0.95,
      temperature=0.9,
      repetition_penalty=1.2,
      no_repeat_ngram_size=3,
      early_stopping=True
    )


    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# بناء واجهة Gradio
interface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=10, placeholder="اكتب النص هنا...", label="📝 النص الكامل"),
    outputs=gr.Textbox(label="✂️ الملخص"),
    title="ملخص نصوص بالعربية باستخدام T5"
)



C:\Users\Admin\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
